# BASELINE VECTOR-ONLY RAG

## Load csv

In [6]:
import pandas as pd

# Load CSV safely
df = pd.read_csv("HRDataset_v14.csv", encoding="utf-8")

print("Columns detected:", list(df.columns))
print("Total rows:", len(df))


Columns detected: ['Employee_Name', 'EmpID', 'MarriedID', 'MaritalStatusID', 'GenderID', 'EmpStatusID', 'DeptID', 'PerfScoreID', 'FromDiversityJobFairID', 'Salary', 'Termd', 'PositionID', 'Position', 'State', 'Zip', 'DOB', 'Sex', 'MaritalDesc', 'CitizenDesc', 'HispanicLatino', 'RaceDesc', 'DateofHire', 'DateofTermination', 'TermReason', 'EmploymentStatus', 'Department', 'ManagerName', 'ManagerID', 'RecruitmentSource', 'PerformanceScore', 'EngagementSurvey', 'EmpSatisfaction', 'SpecialProjectsCount', 'LastPerformanceReview_Date', 'DaysLateLast30', 'Absences']
Total rows: 311


In [7]:
def row_to_text(row):
    parts = []
    for col, val in row.items():
        if pd.notna(val):
            parts.append(f"{col}: {val}")
    return " | ".join(parts)

documents = [row_to_text(row) for _, row in df.iterrows()]

print("Sample document:")
print(documents[0])


Sample document:
Employee_Name: Adinolfi, Wilson  K | EmpID: 10026 | MarriedID: 0 | MaritalStatusID: 0 | GenderID: 1 | EmpStatusID: 1 | DeptID: 5 | PerfScoreID: 4 | FromDiversityJobFairID: 0 | Salary: 62506 | Termd: 0 | PositionID: 19 | Position: Production Technician I | State: MA | Zip: 1960 | DOB: 07/10/83 | Sex: M  | MaritalDesc: Single | CitizenDesc: US Citizen | HispanicLatino: No | RaceDesc: White | DateofHire: 7/5/2011 | TermReason: N/A-StillEmployed | EmploymentStatus: Active | Department: Production        | ManagerName: Michael Albert | ManagerID: 22.0 | RecruitmentSource: LinkedIn | PerformanceScore: Exceeds | EngagementSurvey: 4.6 | EmpSatisfaction: 5 | SpecialProjectsCount: 0 | LastPerformanceReview_Date: 1/17/2019 | DaysLateLast30: 0 | Absences: 1


## Chunking

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

chunks = text_splitter.create_documents(documents)
print("Total chunks created:", len(chunks))



Total chunks created: 992


## Embedding + FAISS

In [9]:
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Embedding model (open-source, local)
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Create in-memory FAISS vector store
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model
)

print("Baseline FAISS vector store ready (in-memory)")


Baseline FAISS vector store ready (in-memory)


## Query & retrieve

In [21]:
# Take query from user input
query = input("Enter your query: ").strip()

if not query:
    raise ValueError("Query cannot be empty")

# Perform similarity search
results = vectorstore.similarity_search(query, k=3)

print("\nRetrieved chunks:\n")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")



Retrieved chunks:

1. | ManagerID: 2.0 | RecruitmentSource: Employee Referral | PerformanceScore: Exceeds | EngagementSurvey: 4.6 | EmpSatisfaction: 5 | SpecialProjectsCount: 6 | LastPerformanceReview_Date: 2/21/2019 | DaysLateLast30: 0 | Absences: 16
2. for Cause | Department: Sales | ManagerName: Lynn Daneault | ManagerID: 21.0 | RecruitmentSource: Employee Referral | PerformanceScore: PIP | EngagementSurvey: 2.0 | EmpSatisfaction: 5 | SpecialProjectsCount: 0 | LastPerformanceReview_Date: 1/28/2019 | DaysLateLast30: 4 | Absences: 7
3. | ManagerID: 7.0 | RecruitmentSource: Employee Referral | PerformanceScore: Fully Meets | EngagementSurvey: 4.11 | EmpSatisfaction: 4 | SpecialProjectsCount: 6 | LastPerformanceReview_Date: 2/25/2019 | DaysLateLast30: 0 | Absences: 16


# KG Construction:

In [17]:
import pandas as pd
from neo4j import GraphDatabase
import math

# -------- Configuration --------
NEO4J_URI = "neo4j+ssc://b931b8dd.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "mW-0f4ShNF394EffXlczpH-5onIZuEsXRFatduQQN5I" 
NEO4J_DATABASE = "neo4j"

# -------- Data Preparation --------
def load_and_clean_data(filepath):
    df = pd.read_csv(filepath)

    # 1. Clean Dates
    date_cols = ["DateofHire", "DateofTermination", "DOB", "LastPerformanceReview_Date"]
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors="coerce")

    # 2. Fix ID Types (Float -> Int -> String)
    # This handles the case where ManagerID is read as 39.0 instead of 39
    df["EmpID"] = df["EmpID"].astype(str)
    
    # Handle nullable ManagerID safely
    def clean_manager_id(x):
        if pd.isna(x):
            return None
        return str(int(x)) # Converts 39.0 -> 39 -> "39"
    
    df["ManagerID"] = df["ManagerID"].apply(clean_manager_id)
    df["PositionID"] = df["PositionID"].apply(lambda x: str(int(x)) if pd.notna(x) else None)

    # 3. Handle Text NaNs
    df.fillna("", inplace=True)
    
    # 4. Convert Date Objects to Strings for Neo4j (Neo4j driver handles ISO strings best)
    # We create a dictionary list for batch ingestion
    records = []
    for _, row in df.iterrows():
        record = {
            "emp_id": row["EmpID"],
            "name": row["Employee_Name"],
            "salary": float(row["Salary"]) if row["Salary"] else 0.0,
            "position_id": row["PositionID"],
            "position_title": row["Position"],
            "dept_id": str(row["DeptID"]),
            "department": row["Department"],
            "manager_id": row["ManagerID"],
            "recruitment_source": row["RecruitmentSource"],
            "performance_score": row["PerformanceScore"],
            "hire_date": row["DateofHire"].date().isoformat() if pd.notna(row["DateofHire"]) else None,
            "term_date": row["DateofTermination"].date().isoformat() if pd.notna(row["DateofTermination"]) else None,
            "dob": row["DOB"].date().isoformat() if pd.notna(row["DOB"]) else None,
            "sex": row["Sex"],
            "race": row["RaceDesc"],
            "marital_status": row["MaritalDesc"]
        }
        records.append(record)
        
    return records

# -------- Cypher Operations --------

def create_constraints(driver):
    """Creates unique constraints to ensure data integrity and speed up MERGE."""
    queries = [
        "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Person) REQUIRE p.emp_id IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (d:Department) REQUIRE d.name IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (pos:Position) REQUIRE pos.title IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (s:Source) REQUIRE s.name IS UNIQUE"
    ]
    with driver.session(database=NEO4J_DATABASE) as session:
        for q in queries:
            session.run(q)
    print("Constraints verified.")

def ingest_batch(driver, data):
    """
    Ingests all data in a single optimized batch transaction using UNWIND.
    """
    query = """
    UNWIND $rows AS row
    
    // 1. Merge Department
    MERGE (d:Department {name: row.department})
    
    // 2. Merge Position
    MERGE (pos:Position {title: row.position_title})
    SET pos.position_id = row.position_id
    
    // 3. Merge Recruitment Source (New Node Type)
    MERGE (src:Source {name: row.recruitment_source})

    // 4. Merge Person (Central Node)
    MERGE (p:Person {emp_id: row.emp_id})
    SET 
        p.name = row.name,
        p.salary = row.salary,
        p.dob = date(row.dob),
        p.sex = row.sex,
        p.race = row.race,
        p.marital_status = row.marital_status,
        p.performance_score = row.performance_score

    // 5. Relationships
    MERGE (p)-[:WORKS_IN]->(d)
    MERGE (p)-[:RECRUITED_FROM]->(src)

    // Handle 'HELD_POSITION' with history logic
    MERGE (p)-[r:HELD_POSITION]->(pos)
    SET 
        r.from = date(row.hire_date)
    // Only set 'to' date if the employee is actually terminated
    FOREACH (_ IN CASE WHEN row.term_date IS NOT NULL THEN [1] ELSE [] END |
        SET r.to = date(row.term_date)
    )
    
    // 6. Manager Relationship
    // We do this inside the same UNWIND using a sub-query match
    with p, row
    WHERE row.manager_id IS NOT NULL AND row.manager_id <> ""
    MATCH (m:Person {emp_id: row.manager_id})
    MERGE (p)-[:REPORTS_TO]->(m)
    """
    
    with driver.session(database=NEO4J_DATABASE) as session:
        session.run(query, rows=data)
    print(f"Batch ingestion complete for {len(data)} records.")


In [18]:

# -------- Main Execution --------

if __name__ == "__main__":
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    
    try:
        # 1. Prepare Data
        print("Processing CSV...")
        data = load_and_clean_data("HRDataset_v14.csv")
        
        # 2. Setup Database
        create_constraints(driver)
        
        # 3. Ingest Data
        # Note: If you have >10,000 rows, chunk 'data' into batches of 1000
        print("Ingesting Graph...")
        ingest_batch(driver, data)
        
        print("Success! Temporal Knowledge Graph created.")
        
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.close()

Processing CSV...


C:\Users\admin\AppData\Local\Temp\ipykernel_776\2907144151.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")


Constraints verified.
Ingesting Graph...
Batch ingestion complete for 311 records.
Success! Temporal Knowledge Graph created.
